In [1]:
# Install the required Python packages using pip
# The -q flag suppresses the output to keep the installation process quiet
!pip -q install google-generativeai gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 73.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.0 MB/s eta 0:00:00


In [2]:
# Importing necessary libraries
import os  # for interacting with the operating system, e.g., file paths
from openai import OpenAI  # to interact with OpenAI's API
import google.generativeai as genai  # to interact with Google's generative AI API
import gradio as gr  # for building a user

In [3]:
# Setting up Google API key to authenticate with Google Generative AI
GOOGLE_API_KEY = "Gemini API Key"  # Replace with your own API key
genai.configure(api_key=GOOGLE_API_KEY)  # Configure Google Generative AI with the provided API key

In [4]:
# Define a system prompt for the AI model, outlining its role and response behavior
system_prompt = (
    "You are a financial expert who guides people in managing their finances and helps them in budgeting and planning their expenses, "
    "so people can keep track of their finances. If you are unsure about the information, just say 'You are unsure.' "
    "Also, take feedback for the planning system provided. Ask the user if they want further suggestions."
)
# The system prompt sets the tone and behavior of the AI, instructing it to focus on finance, be transparent when unsure,
# and encourage user feedback and further interaction.m

In [5]:
# Function to send a prompt to the Gemini model and receive a response
def message_gemini(prompt):
    # Prepare the messages with roles (system and user) for the prompt
    messages = [
        {"role": "system", "content": system_prompt},  # System message to set context for the AI
        {"role": "user", "content": prompt}  # User's prompt, which is the input query
    ]

    # Initialize an empty string to accumulate the full prompt
    full_prompt = ""

    # Loop through each message in the messages list to construct the full prompt
    for msg in messages:
        role = msg["role"]  # Get the role (either system or user)
        content = msg["content"]  # Get the content of the message

        # Append the system message directly
        if role == "system":
            full_prompt += f"{content}\n"
        # Format user messages with a "User:" prefix
        elif role == "user":
            full_prompt += f"User: {content}\n"

    # Initialize the model using the specified model name "gemini-2.0-flash-exp"
    model = genai.GenerativeModel("gemini-2.0-flash-exp")

    # Generate content using the full_prompt as input
    response = model.generate_content(full_prompt)

    # Return the generated response text from the model
    return response.text  # The generated content (response) from Gemini AI model


In [6]:
# Call the message_gemini function with the user input "Hello"
# The function constructs a full prompt with the system context and the user message ("Hello")
# It then sends this prompt to the Gemini model and retrieves the AI-generated response.
response = message_gemini("Hello")
response
# The response variable will contain the AI's generated response to the "Hello" message.


"Hello! I'm happy to help you get a better handle on your finances. I can guide you through budgeting, expense tracking, and financial planning.\n\nTo get started, tell me a little bit about your current financial situation. For example:\n\n*   **What are your financial goals?** (e.g., saving for a down payment, paying off debt, investing for retirement)\n*   **Do you currently track your income and expenses?** If so, how?\n*   **What are your biggest financial concerns right now?**\n*   **What is your approximate monthly income and expenses?** (Don't need exact figures, just ballpark estimates)\n\nThe more information you provide, the better I can tailor my advice to your specific needs. Let's work together to achieve your financial goals!\n"

In [7]:
# Function to stream the response from the Gemini API as the content is generated
def stream_gemini(prompt):
    # Define the messages to be sent to the Gemini model, including system context and user input
    messages = [
        {"role": "system", "content": system_prompt},  # System message setting the behavior of the model
        {"role": "user", "content": prompt},  # User's input query that will be processed by the model
    ]

    # Initialize the OpenAI API client for Google Generative AI with the necessary API key and endpoint
    gemini_api = OpenAI(
        api_key=GOOGLE_API_KEY,  # API key for authentication
        base_url="https://generativelanguage.googleapis.com/v1beta/openai/",  # API endpoint URL
    )

    # Request a streaming completion from the Gemini model
    response = gemini_api.chat.completions.create(
        model="gemini-2.0-flash-exp",  # Specify the model to be used
        messages=messages,  # Pass the constructed messages to the API
        stream=True,  # Enable streaming for real-time response generation
    )

    # Initialize an empty string to accumulate the results as chunks are received
    result = ""

    # Loop through each chunk in the streaming response
    for chunk in response:
        # Extract the content from the chunk's choice and append it to the result
        result += chunk.choices[0].delta.content or ""  # If there's no content, it appends an empty string
        # Yield the accumulated result so that it can be processed incrementally
        yield result


In [8]:
# 🔹 Gradio UI setup for the financial advisor

# Create a Gradio interface to interact with the stream_gemini function
with gr.Interface(
    fn=stream_gemini,  # Function to be called when user interacts with the UI (streaming response from Gemini)
    inputs=[gr.Textbox(label="Your Response:")],  # Textbox input for the user to enter their response/query
    outputs=[gr.Markdown(label="Response:")],  # Markdown output area to display the generated response
    allow_flagging="never"  # Disable the flagging feature in the UI (users can't flag content)
) as view:
    # Launch the Gradio interface to make it accessible to the user
    view.launch()  # This will start the UI and allow the user to interact with it


/usr/local/lib/python3.11/dist-packages/gradio/interface.py:415: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://16ee90c04009cfb743.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [9]:
# Function to handle a chat session with Gemini, including maintaining conversation history
def chat_gemini(message, history):
    # Construct the list of messages to send to the Gemini model
    # Combine the system prompt, previous chat history, and the current user message
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]

    # Initialize the OpenAI API client with the provided API key and endpoint
    gemu = OpenAI(
        api_key=GOOGLE_API_KEY,  # Authentication using the Google API key
        base_url="https://generativelanguage.googleapis.com/v1beta/openai/",  # The API endpoint URL
    )

    # Make a streaming request to the Gemini model for chat completion
    stream = gemu.chat.completions.create(
        model="gemini-2.0-flash-exp",  # Specify the model to use for generating the response
        messages=messages,  # Pass the combined messages (including history and current user input)
        stream=True  # Enable streaming to get the response incrementally
    )

    # Initialize an empty string to accumulate the response as it streams in
    response = ""

    # Loop through each chunk of the streamed response
    for chunk in stream:
        # Append the chunk's content to the response string (if any content is present)
        response += chunk.choices[0].delta.content or ''

        # Yield the current response (this allows the response to be processed incrementally)
        yield response


In [10]:
# Gradio Chat Interface for interacting with the Gemini model

gr.ChatInterface(
    fn=chat_gemini,  # The function that handles the chat, which is 'chat_gemini'
    type="messages",  # Type of interface to be used - in this case, a message-based chat
    theme=gr.themes.Base(primary_hue="emerald"),  # Setting the theme of the UI with a primary color hue (emerald)

    # Custom CSS for styling the interface (dark theme)
    css="""
    body {
        background-color: black;
        color: white;
        font-family: Arial, sans-serif;
    }
    .gr-chatbot {
        background-color: #0f172a;
        border-radius: 15px;
        padding: 10px;
    }  # Style the chat window with a dark background

    textarea, .gr-button {
        background-color: #1a1a1a;
        color: white;
        border-radius: 9999px;
    }  # Style buttons and textareas

    .gradio-title {
        font-size: 24px;
        text-align: center;
        color: #0fdf6e;
        margin-bottom: 20px;
    }  # Style for the heading 'Financial Advisor AI'
    """,
    # Adding the heading as a title to the interface
    title="Financial Advisor AI",  # The title displayed at the top of the Gradio interface
).launch(share=True)  # Launch the interface with sharing enabled (makes the app publicly accessible)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://175f5b5e78bdf18197.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [31]:
!apt-get install git
!apt-get install openssh-client

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.12).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
openssh-client is already the newest version (1:8.9p1-3ubuntu0.11).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [32]:
!ssh-keygen -t rsa -b 4096 -C "shamithar.bsc23@rvu.edu.in"

Generating public/private rsa key pair.
Enter file in which to save the key (/root/.ssh/id_rsa): 
/root/.ssh/id_rsa already exists.
Overwrite (y/n)? y
Enter passphrase (empty for no passphrase): 
Enter same passphrase again: 
Your identification has been saved in /root/.ssh/id_rsa
Your public key has been saved in /root/.ssh/id_rsa.pub
The key fingerprint is:
SHA256:HIHdlJckmQkrlFJIy3EPwwFRfQfk8H0123NbU6nFrXA shamithar.bsc23@rvu.edu.in
The key's randomart image is:
+---[RSA 4096]----+
|   .=BO*+==B....=|
|   .o=++o*Bo= E=*|
|    o.. +.oo.o+=+|
|       o .   o. *|
|        S      . |
|                 |
|                 |
|                 |
|                 |
+----[SHA256]-----+


In [33]:
!eval $(ssh-agent -s)
!ssh-add /root/.ssh/id_rsa

Agent pid 21600
Could not open a connection to your authentication agent.


In [34]:
!cat /root/.ssh/id_rsa.pub

ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAACAQC4W4YJRS0zx8bm/N4OmD+a9tUEoJvViEfaa95MlykANgJa2vOum4/f7a834JwYuHcGBa8AdBVaJFjNbAc7V63AT6YS20ASvRN3ecCm3AvLKxgBrx2bfNu3HuCejrv4d1coMIgDWuqSTop0e292nsRh3kVmOJilTbVxWkS4haNlGiBg/qB1lsNu1Akr1ITEbquxqK8NFugPPisUVtuL/WAcLxWo24phMoTvGq4+mG2iuErzDOooL5m2TnKkmd9W/G94FiPS9axDu38ZPL4n8KjjUZCTIqEQcWpj7rhqDTXlEYBgWJztZvGuVrcREkBvPmfAU9PP4LvHSmhkJrIvH9ypYke1mVSviAtbzJxnakZNwJRCUpQ1Y926RedI+pgb2EixMk+neQZCMOIKqLzF4p8xJ6GLqllVsSSruqwPUVh0u0Lpo1LUiH23YdCIueJU96pvTwtzFEbB8ZHeqlVQ0i61h1H6/fhFtXQL7I4yyS+I73yVuOemCh0OAcYtkJ4dC2FMt4bLnc33LSHiQP/voFkKldgYrE2msFBQOEThI3G8YJSu7loSfIteEKOvXh6gqSF334Mqi0LcvKF9jIBTn9JgPOvz3mV/FsP7grjDEuNsbo98ClttPba7z/EA2ywg5k7R0rTt87Hm6EjbGuUn041Q2FnxqMPZ/szTck2RxwR+xw== shamithar.bsc23@rvu.edu.in


In [35]:
!ssh -T git@github.com

Hi Shamithabsc! You've successfully authenticated, but GitHub does not provide shell access.


In [36]:
!git clone git@github.com:Shamithabsc/Personal_FinanceAdvisor-GenAI.git

fatal: destination path 'Personal_FinanceAdvisor-GenAI' already exists and is not an empty directory.


In [37]:
%cd Personal_FinanceAdvisor-GenAI

/content/Personal_FinanceAdvisor-GenAI


In [38]:
!cp /content/Finance_Advisor.py

cp: missing destination file operand after '/content/Finance_Advisor.py'
Try 'cp --help' for more information.


In [39]:
!cp /content/Finance_Advisor.ipynb

cp: missing destination file operand after '/content/Finance_Advisor.ipynb'
Try 'cp --help' for more information.


In [40]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [46]:
!ls /content/drive/My\ Drive/Colab_Notebooks/Finance_Advisor.ipynb

'/content/drive/My Drive/Colab_Notebooks/Finance_Advisor.ipynb'


In [50]:
!cp "/content/drive/My\ Drive/Colab_Notebooks/Finance_Advisor.ipynb" /content/Personal_FinanceAdvisor-GenAI/


cp: cannot stat '/content/drive/My\ Drive/Colab_Notebooks/Finance_Advisor.ipynb': No such file or directory
